### Approach
* Build a static relationship list containing the most common relationship names
* Sentence segmentation
* Tokenization
* Search for names in the sentence


In [2]:
import spacy
import pprint
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.tag import StanfordNERTagger

nlp = spacy.load('de')
#nlp = spacy.load('de_core_news_sm')
#form fuzzywuzzy import fuzz

model = '../models/dewac_175m_600.crf.ser.gz'
#model = '../models/hgc_175m_600.crf.ser.gz'
#model = '../models/german.conll.germeval2014.hgc_175m_600.crf.ser.gz'

st = StanfordNERTagger(model,
                       '../models/stanford-ner.jar',
                       encoding='utf-8')

c:\program files\python\lib\site-packages\nltk\tag\stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [31]:
text_long = u'''Meine Enkelin Lisa und mein Enkel Lukas fliegen morgen nach London. Sie sind zum ersten Mal in England.
Peter und Maria gehen morgen ins Kino. Ich und mein Sohn gehen heute zum Fußball. Ich bin geboren zu York im Jahre 1632, als Kind angesehener Leute, die ursprünglich nicht aus jener Gegend stammten. 
Mein Vater, ein Ausländer, aus Bremen gebürtig, hatte sich zuerst in Hull niedergelassen, war dort als Kaufmann zu 
hübschem Vermögen gekommen und dann, nachdem er sein Geschäft aufgegeben hatte, nach York gezogen. 
Hier heiratete er meine Mutter, eine geborene Robinson.
Ich hatte zwei ältere Brüder. Der eine von ihnen, welcher als Oberstleutnant bei einem englischen, 
früher von dem berühmten Oberst Lockhart befehligten Infanterieregiment in Flandern diente, 
fiel in der Schlacht bei Dünkirchen. Was aus dem jüngeren geworden ist, habe ich ebenso wenig in Erfahrung bringen können, 
als meine Eltern je Kenntnisse von meinen eignen Schicksalen erhalten haben.'''

In [3]:
text = u'''Meine Enkelin Lisa und mein Enkel Lukas fliegen morgen nach London. Sie sind zum ersten Mal in England. 
Peter und Maria gehen morgen ins Kino. Ich und mein Sohn gehen heute zum Fußball.'''

In [4]:
relationship_list = ['vater', 'mutter', 'sohn', 'tochter', 'bruder', 'schwester', 'enkel', 'enkelin', 'nichte', 
                     'neffe', 'onkel', 'tante']

In [5]:
me_list = ['ich', 'meine', 'mein']

## POS Tagging

In [8]:
doc = nlp(text)

pos_tagged_sentences = []
for token in doc:
    pos_tuple = (token.text, token.pos_)
    pos_tagged_sentences.append(pos_tuple)
    print(token.text, token.pos_, token.dep_,)
    
pos_tagged_sentences

Meine DET nk
Enkelin PROPN pnc
Lisa PROPN sb
und CONJ cd
mein DET nk
Enkel NOUN cj
Lukas PROPN ag
fliegen VERB ROOT
morgen ADV mo
nach ADP mo
London PROPN nk
. PUNCT punct
Sie PRON sb
sind AUX ROOT
zum ADP mo
ersten ADJ nk
Mal NOUN nk
in ADP mo
England PROPN nk
. PUNCT punct

 SPACE 
Peter PROPN sb
und CONJ cd
Maria PROPN cj
gehen VERB ROOT
morgen ADV mo
ins ADP mo
Kino NOUN nk
. PUNCT punct
Ich PRON sb
und CONJ cd
mein DET nk
Sohn NOUN cj
gehen VERB ROOT
heute ADV mo
zum ADP mo
Fußball NOUN nk
. PUNCT punct


[('Meine', 'DET'),
 ('Enkelin', 'PROPN'),
 ('Lisa', 'PROPN'),
 ('und', 'CONJ'),
 ('mein', 'DET'),
 ('Enkel', 'NOUN'),
 ('Lukas', 'PROPN'),
 ('fliegen', 'VERB'),
 ('morgen', 'ADV'),
 ('nach', 'ADP'),
 ('London', 'PROPN'),
 ('.', 'PUNCT'),
 ('Sie', 'PRON'),
 ('sind', 'AUX'),
 ('zum', 'ADP'),
 ('ersten', 'ADJ'),
 ('Mal', 'NOUN'),
 ('in', 'ADP'),
 ('England', 'PROPN'),
 ('.', 'PUNCT'),
 ('\n', 'SPACE'),
 ('Peter', 'PROPN'),
 ('und', 'CONJ'),
 ('Maria', 'PROPN'),
 ('gehen', 'VERB'),
 ('morgen', 'ADV'),
 ('ins', 'ADP'),
 ('Kino', 'NOUN'),
 ('.', 'PUNCT'),
 ('Ich', 'PRON'),
 ('und', 'CONJ'),
 ('mein', 'DET'),
 ('Sohn', 'NOUN'),
 ('gehen', 'VERB'),
 ('heute', 'ADV'),
 ('zum', 'ADP'),
 ('Fußball', 'NOUN'),
 ('.', 'PUNCT')]

In [7]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children])

Meine nk Lisa PROPN []
Enkelin pnc Lisa PROPN []
Lisa sb fliegen VERB [Meine, Enkelin, und]
und cd Lisa PROPN [Enkel]
mein nk Enkel NOUN []
Enkel cj und CONJ [mein, Lukas]
Lukas ag Enkel NOUN []
fliegen ROOT fliegen VERB [Lisa, morgen, nach, .]
morgen mo fliegen VERB []
nach mo fliegen VERB [London]
London nk nach ADP []
. punct fliegen VERB []
Sie sb sind AUX []
sind ROOT sind AUX [Sie, zum, in, .]
zum mo sind AUX [Mal]
ersten nk Mal NOUN []
Mal nk zum ADP [ersten]
in mo sind AUX [England]
England nk in ADP []
. punct sind AUX [
]

  . PUNCT []
Peter sb gehen VERB [und]
und cd Peter PROPN [Maria]
Maria cj und CONJ []
gehen ROOT gehen VERB [Peter, morgen, ins, .]
morgen mo gehen VERB []
ins mo gehen VERB [Kino]
Kino nk ins ADP []
. punct gehen VERB []
Ich sb gehen VERB [und]
und cd Ich PRON [Sohn]
mein nk Sohn NOUN []
Sohn cj und CONJ [mein]
gehen ROOT gehen VERB [Ich, heute, zum, .]
heute mo gehen VERB []
zum mo gehen VERB [Fußball]
Fußball nk zum ADP []
. punct gehen VERB []


### Extract all sentences containing two or more named entities

##### Using spaCy NER  DEPRECATED

In [22]:
sentences_to_check = []
labeled_sentences = []
for sentence in sent_tokenize(text):
    labeled_sentence = []
    doc = nlp(sentence)
    for ent in doc.ents:
        tuple = (ent.text, ent.label_)
        labeled_sentence.append((ent.text, ent.label_))
    
    print(labeled_sentence)
    for label in labeled_sentences:
        if 'PER' in label[1]:
            sentences_to_check.append(labeled_sentence)

sentences_to_check

[('Meine Enkelin Lisa', 'MISC'), ('Lukas', 'PER'), ('London', 'LOC')]
[('England', 'LOC')]
[('Ich bin geboren zu York im Jahre 1632', 'MISC')]
[('Mein', 'PER'), ('Bremen', 'LOC'), ('Hull', 'LOC'), ('\nhübschem Vermögen', 'PER'), ('York', 'LOC')]
[('Robinson', 'PER')]
[('Brüder', 'MISC')]
[('englischen', 'MISC'), ('\n', 'MISC'), ('Lockhart', 'PER'), ('Flandern', 'LOC'), ('\n', 'MISC'), ('Schlacht bei Dünkirchen', 'MISC')]
[('\n', 'MISC')]


[]

##### Using Stanford NER

In [40]:
sentences_to_check = []


for sentence in sent_tokenize(text):
    ner_tuples = st.tag(sentence.split())
    rel_tuples = []
    label = 0
    for ner_tuple in ner_tuples:        
        if 'I-PER' in ner_tuple:
            label += 1
            tuple = (ner_tuple[0], 'PER')
            rel_tuples.append(tuple)
        elif ner_tuple[0].lower() in relationship_list:
            label += 1
            tuple = (ner_tuple[0], 'REL')
            rel_tuples.append(tuple)
        elif ner_tuple[0].lower() in me_list:
            label += 1
            tuple = (ner_tuple[0], 'PME')
            rel_tuples.append(tuple)
    
    if label >= 2:
        sentences_to_check.append(rel_tuples)

sentences_to_check

#for sentence in sentences_to_check:
#    tokenized_text = word_tokenize(utterance)
#    classified = st.tag(tokenized_text)
    
#classified

[[('Meine', 'PME'), ('Enkelin', 'REL'), ('Lisa', 'PER'), ('Enkel', 'REL'), ('Lukas', 'PER')], [('Peter', 'PER'), ('Maria', 'PER')], [('ich', 'PME'), ('meine', 'PME')]]


[('ich', 'PME'), ('meine', 'PME')]

 ### Extract Features

In [29]:
for sentence in sentences_to_check:
    

[[('Meine', 'O'),
  ('Enkelin', 'O'),
  ('Lisa', 'I-PER'),
  ('und', 'O'),
  ('mein', 'O'),
  ('Enkel', 'O'),
  ('Lukas', 'I-PER'),
  ('fliegen', 'O'),
  ('morgen', 'O'),
  ('nach', 'O'),
  ('London.', 'I-LOC')],
 [('Der', 'O'),
  ('eine', 'O'),
  ('von', 'O'),
  ('ihnen,', 'O'),
  ('welcher', 'O'),
  ('als', 'O'),
  ('Oberstleutnant', 'O'),
  ('bei', 'O'),
  ('einem', 'O'),
  ('englischen,', 'O'),
  ('früher', 'O'),
  ('von', 'O'),
  ('dem', 'O'),
  ('berühmten', 'O'),
  ('Oberst', 'O'),
  ('Lockhart', 'I-PER'),
  ('befehligten', 'O'),
  ('Infanterieregiment', 'O'),
  ('in', 'O'),
  ('Flandern', 'I-LOC'),
  ('diente,', 'O'),
  ('fiel', 'O'),
  ('in', 'O'),
  ('der', 'O'),
  ('Schlacht', 'O'),
  ('bei', 'O'),
  ('Dünkirchen.', 'I-LOC')]]

In [38]:
for sentence in sentences_to_check:
    doc = nlp(sentence)
    print(sentence)
    for ent in doc.ents:
        print(ent.text, ent.label_)

Meine Enkelin Lisa und mein Enkel Lukas fliegen morgen nach London.
Meine Enkelin Lisa MISC
Lukas PER
London LOC
Meine Enkelin Lisa und mein Enkel Lukas fliegen morgen nach London.
Meine Enkelin Lisa MISC
Lukas PER
London LOC
Mein Vater, ein Ausländer, aus Bremen gebürtig, hatte sich zuerst in Hull niedergelassen, war dort als Kaufmann zu 
hübschem Vermögen gekommen und dann, nachdem er sein Geschäft aufgegeben hatte, nach York gezogen.
Mein PER
Bremen LOC
Hull LOC

hübschem Vermögen PER
York LOC
Hier heiratete er meine Mutter, eine geborene Robinson.
Robinson PER


## Stanford Dependency Parser

In [11]:
from nltk.parse.stanford import StanfordDependencyParser

path_to_jar = '../models/stanford-parser.jar'
path_to_models_jar = '../models/stanford-german-corenlp-models.jar'

dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)

result = dependency_parser.raw_parse_sents('Ich habe eine Enkelin namens Lisa.')
dep = result.__next__()

result
#list(dep.triples())


### spaCy Dependency Parser

In [4]:
doc = nlp(text)
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children])

Meine nk Lisa PROPN []
Enkelin pnc Lisa PROPN []
Lisa sb fliegen VERB [Meine, Enkelin, und]
und cd Lisa PROPN [Enkel]
mein nk Enkel NOUN []
Enkel cj und CONJ [mein, Lukas]
Lukas ag Enkel NOUN []
fliegen ROOT fliegen VERB [Lisa, morgen, nach, .]
morgen mo fliegen VERB []
nach mo fliegen VERB [London]
London nk nach ADP []
. punct fliegen VERB []
Sie sb sind AUX []
sind ROOT sind AUX [Sie, zum, in, .]
zum mo sind AUX [Mal]
ersten nk Mal NOUN []
Mal nk zum ADP [ersten]
in mo sind AUX [England]
England nk in ADP []
. punct sind AUX []


In [5]:
print('\n'.join('{child:<8} <{label:-^7} {head}'.format(child=t.orth_, label=t.dep_, head=t.head.orth_) for t in doc))

Meine    <--nk--- Lisa
Enkelin  <--pnc-- Lisa
Lisa     <--sb--- fliegen
und      <--cd--- Lisa
mein     <--nk--- Enkel
Enkel    <--cj--- und
Lukas    <--ag--- Enkel
fliegen  <-ROOT-- fliegen
morgen   <--mo--- fliegen
nach     <--mo--- fliegen
London   <--nk--- nach
.        <-punct- fliegen
Sie      <--sb--- sind
sind     <-ROOT-- sind
zum      <--mo--- sind
ersten   <--nk--- Mal
Mal      <--nk--- zum
in       <--mo--- sind
England  <--nk--- in
.        <-punct- sind
